***Genetic Algorithms(GAs)***

* Genetic Algorithms(GAs) are adaptive heuristic search algorithms that belong to the larger part of evolutionary algorithms. Genetic algorithms are based on the ideas of natural selection and genetics. These are intelligent exploitation of random search provided with historical data to direct the search into the region of better performance in solution space.

* Genetic algorithms simulate the process of natural selection
* Each generation consist of a population of individuals and each individual represents a point in search space and possible solution. 
* Each individual is represented as a string of character/integer/float/bits. This string is analogous to the Chromosome.

**Search space**
* The population of individuals are maintained within search space.
* Each individual represents a solution in search space for given problem.
* Each individual is coded as a finite length vector (analogous to chromosome) of components.
* These variable components are analogous to Genes. Thus a chromosome (individual) is composed of several genes (variable components).

**Fitness Score**
* A Fitness Score is given to each individual which shows the ability of an individual to “compete”.
* The individual having optimal fitness score (or near optimal) are sought.
* The GAs maintains the population of n individuals (chromosome/solutions) along with their fitness scores.
* The individuals having better fitness scores are given more chance to reproduce than others.
* The population size is static so the room has to be created for new arrivals.
* So, some individuals die and get replaced by new arrivals eventually creating new generation when all the mating opportunity of the old population is exhausted.
* It is hoped that over successive generations better solutions will arrive while least fit die.
* Each new generation has on average more “better genes” than the individual (solution) of previous generations.
* Once the offspring produced having no significant difference from offspring produced by previous populations, the population is converged. The algorithm is said to be converged to a set of solutions for the problem.

**Operators of Genetic Algorithms**
* Once the initial generation is created, the algorithm evolves the generation using following operators

**1) Selection Operator:**
* The idea is to give preference to the individuals with good fitness scores and allow them to pass their genes to successive generations.

**2) Crossover Operator:**
* This represents mating between individuals. Two individuals are selected using selection operator and crossover sites are chosen randomly.
* Then the genes at these crossover sites are exchanged thus creating a completely new individual (offspring).

**3) Mutation Operator:**
* The key idea is to insert random genes in offspring to maintain the diversity in the population to avoid premature convergence.

1) Randomly initialize populations p
2) Determine fitness of population
3) Until convergence repeat:
a) Select parents from population
b) Crossover and generate new population
c) Perform mutation on new population
d) Calculate fitness for new population

*Activity 1:*
* Consider maxOne problem where the goal is to arrange a string of L bits into all ones. At first the solution may seem trivial i.e., for L=8 the solution is [1, 1, 1, 1, 1, 1, 1, 1]. Despite this we shall see how many iterations it will take for an instance of genetic algorithm to find the solution.

We start by looking at the GA class which is instantiated by providing individualSize (length of chromosome or L in our case) and populationSize (how many survivors will we retain in each generation). The instance variable population is a dictionary of items where each item contains a bit array of an individual and its heuristic distance from the goal. The variable totalFitness maintains the total heuristic value of each generation/population.

In [1]:
import random
import math

class GA:
    def __init__(self, individualSize, populationSize):
        # Initialize population dictionary to store individuals
        self.population = dict()  
        # Size of each individual's chromosome (number of bits)
        self.individualSize = individualSize
        # Number of individuals in the population
        self.populationSize = populationSize
        # Total fitness of the entire population
        self.totalFitness = 0

        # Create initial population
        i=0
        while i<populationSize:
            # Create a list of bits initialized to 0, i.e., [0, 0, 0, 0, 0, 0, 0, 0]
            listOfBits = [0] * individualSize
            # Create list of all possible bit positions i.e., [0, 1, 2, 3, 4, 5, 6, 7]
            listOfLocations = list(range(0, individualSize))
            # Randomly decide how many 1s this individual will have
            numberOfOnes = random.randint(0, individualSize-1)
            # Randomly select positions for the 1s, i.e., if numberOfOnes = 3, it might select [2, 5, 7] 
            # .sample() randomly selects (numberofones) unique elements from the (listoflocations)).
            onesLocations = random.sample(listOfLocations, numberOfOnes)
            # Set selected positions to 1
            for j in onesLocations:
                listOfBits[j] = 1

            # Store individual in population: [chromosome, fitness], whereas key is index of individual and value is a list of two elements.
            # e.g., [ [0, 1, 0, 1, 0, 0, 1, 0], 3 ]
            self.population[i] = [listOfBits, numberOfOnes]
            # Update total fitness
            self.totalFitness = self.totalFitness + numberOfOnes
            i = i + 1

    def updatePopulationFitness(self):
        """Recalculates the fitness of the entire population"""
        self.totalFitness = 0
        for individual in self.population:
            # Calculate fitness as sum of bits (count of 1s in the chromosome (bitstring of individual))
            individualFitness = sum(self.population[individual][0])
            # The fitness is stored as the second element in the individual's value (first is the chromosome).
            # Update individual's fitness value in the population dictionary
            self.population[individual][1] = individualFitness
            # Add to total fitness
            self.totalFitness = self.totalFitness + individualFitness

    def selectParents(self):
        """Selects parents using roulette wheel selection"""
        rouletteWheel = []
        # Determine size of roulette wheel (5x population size)
        wheelSize = self.populationSize * 5
        # List to store all fitness values
        h_n = []
        # Collect all fitness values
        for individual in self.population:
            h_n.append(self.population[individual][1])
        
        # Build roulette wheel
        j = 0
        for individual in self.population:
            # Calculate proportional length on wheel for this individual
            individualLength = round(wheelSize*(h_n[j]/sum(h_n)))
            # lets say Total fitness = sum(h_n) = 10, wheelSize = 20 So for individual 0 with fitness 3:
            # individualLength = round(20 * (3/10)) = round(6.0) = 6
            # This means individual 0 will occupy 6 slots on the wheel.
            j=j+1
            # Append individual to wheel based on its fitness
            if individualLength > 0:
                # If individual 0 has individualLength = 6, then: rouletteWheel = [0, 0, 0, 0, 0, 0] Over all individuals,
                # final rouletteWheel might be: [0,0,0,0,0,0,1,1,1,1,1,1,2,3,3,3,3,3,3,3]
                i=0
                while i<individualLength:
                    rouletteWheel.append(individual)
                    i=i+1
        
        # Shuffle the wheel for random selection
        random.shuffle(rouletteWheel)
        parentIndices = []
        # Select parents from roulette wheel
        i=0
        while i<self.populationSize:   #If populationSize = 4, then you select 4 parents.
            # Select random index from the roulette wheel
            parentIndices.append(rouletteWheel[random.randint(0, len(rouletteWheel)-1)])
            i=i+1
        
        # Create new generation from selected parents
        newGeneration = dict()
        i=0
        while i<self.populationSize:
            newGeneration[i] = self.population[parentIndices[i]].copy()
            # If parentIndices = [1, 0, 3, 1], then new generation will be made of copies of those individuals.
            i=i+1
        
        # Replace old population with new generation
        del self.population
        self.population = newGeneration.copy()

        # Update fitness values for new population
        self.updatePopulationFitness()

    def generateChildren(self, crossoverprobability):   # performs crossover on a subset of individuals in the population based on a specified crossover probability.
        """Performs crossover to create new offspring"""
        # Calculate number of pairs to crossover
        numberofPairs = round(crossoverprobability * self.populationSize / 2)
        # If crossoverprobability = 0.8 and populationSize = 10 Then, numberofPairs = round(0.8 * 10 / 2) = round(4.0) = 4

        # Create list of individual indices and shuffle
        individualIndices = list(range(0, self.populationSize))
        random.shuffle(individualIndices)
        
        i=0     # i counts how many individuals have been processed (in pairs)
        j=0     # j tracks the index in the population used to select two parents (j and j+1)
        # Loop through the shuffled indices in pairs
        while i<numberofPairs:
            # Select random crossover point
            crossoverPoint = random.randint(0, self.individualSize-1)
            # Create children by swapping genetic material
            child1 = self.population[j][0][0:crossoverPoint] + self.population[j+1][0][crossoverPoint:]
            child2 = self.population[j+1][0][0:crossoverPoint] + self.population[j][0][crossoverPoint:]
            #   Example:If individual size is 8 and crossover point is 5:
            #   Child1 = Parent1[0:5] + Parent2[5:] = [1, 0, 1, 1, 0] + [0, 1, 1]
            #   Child2 = Parent2[0:5] + Parent1[5:] = [0, 1, 0, 1, 1] + [1, 0, 0]

            # Replace parents(j,j+1) with children
            # Also update fitness (count of 1s)
            self.population[j] = [child1, sum(child1)]
            self.population[j+1] = [child2, sum(child2)]
            # move to next pair of parents
            i=i+2
            j=j+2

        # Update fitness values after crossover: Each individual's fitness, and The total population fitness
        self.updatePopulationFitness()


    def mutateChildren(self, mutationProbability):
        """Mutates one bit in each individual with a given probability"""
        for i in range(self.populationSize):
            # Randomly decide whether to mutate this individual
            if random.random() < mutationProbability: #random.random() returns a float between 0 and 1. If it’s less than the mutation probability (e.g., 0.1), mutation happens.
                # Pick a random bit to flip
                bitIndex = random.randint(0, self.individualSize - 1)
                # Flip the bit
                if self.population[i][0][bitIndex] == 0:
                    self.population[i][0][bitIndex] = 1
                else:
                    self.population[i][0][bitIndex] = 0
        # Update fitness values after mutation
        self.updatePopulationFitness()

'''
    def mutateChildren(self, mutationProbability):
        """Performs random mutations on the population"""
        # Calculate number of bits to mutate
        numberOfBits = round(mutationProbability * self.populationSize *self.individualSize)
        # Create list of all possible bit positions
        totalIndices = list(range(0, self.populationSize*self.individualSize))
        random.shuffle(totalIndices)
        # Select random bits to mutate
        swapLocations = random.sample(totalIndices, numberOfBits)

        # Perform mutations
        for loc in swapLocations:
            # Calculate which individual and which bit to mutate
            individualIndex = math.floor(loc/self.individualSize)
            bitIndex = math.floor(loc % self.individualSize)
            # Flip the bit (0->1 or 1->0)
            if self.population[individualIndex][0][bitIndex] == 0:
                self.population[individualIndex][0][bitIndex] = 1
            else:
                self.population[individualIndex][0][bitIndex] = 0
        # Update fitness values after mutation 
        self.updatePopulationFitness()
'''     


'\n    def mutateChildren(self, mutationProbability):\n        """Performs random mutations on the population"""\n        # Calculate number of bits to mutate\n        numberOfBits = round(mutationProbability * self.populationSize *self.individualSize)\n        # Create list of all possible bit positions\n        totalIndices = list(range(0, self.populationSize*self.individualSize))\n        random.shuffle(totalIndices)\n        # Select random bits to mutate\n        swapLocations = random.sample(totalIndices, numberOfBits)\n\n        # Perform mutations\n        for loc in swapLocations:\n            # Calculate which individual and which bit to mutate\n            individualIndex = math.floor(loc/self.individualSize)\n            bitIndex = math.floor(loc % self.individualSize)\n            # Flip the bit (0->1 or 1->0)\n            if self.population[individualIndex][0][bitIndex] == 0:\n                self.population[individualIndex][0][bitIndex] = 1\n            else:\n          

In [2]:
# Main execution
individualSize, populationSize = 8, 10
i = 0
# Create GA instance
instance = GA(individualSize, populationSize)
while True:
    # Run one generation of GA
    instance.selectParents()
    instance.generateChildren(0.8)  # 80% crossover probability
    instance.mutateChildren(0.03)   # 3% mutation probability
    # Print current population and fitness
    print(instance.population)
    print(instance.totalFitness)
    print(i)
    i=i+1
    # Check for perfect solution (all 1s)
    found = False
    for individual in instance.population:
        if instance.population[individual][1] == individualSize:
            found = True
            break
    if found:
        break

{0: [[1, 0, 1, 1, 1, 1, 1, 1], 7], 1: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 2: [[0, 0, 0, 0, 0, 0, 0, 1], 1], 3: [[0, 1, 0, 1, 1, 1, 1, 1], 6], 4: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 5: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 6: [[0, 0, 1, 0, 0, 0, 1, 1], 3], 7: [[1, 0, 1, 1, 1, 1, 1, 1], 7], 8: [[1, 0, 1, 1, 0, 1, 0, 0], 4], 9: [[1, 0, 1, 1, 0, 1, 0, 0], 4]}
47
0
{0: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 1: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 2: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 3: [[0, 1, 0, 1, 1, 1, 1, 1], 6], 4: [[1, 0, 1, 1, 0, 1, 0, 0], 4], 5: [[1, 0, 1, 1, 1, 1, 1, 1], 7], 6: [[1, 0, 1, 1, 1, 1, 1, 1], 7], 7: [[1, 0, 1, 1, 0, 1, 0, 0], 4], 8: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 9: [[0, 0, 0, 1, 1, 1, 1, 1], 5]}
53
1
{0: [[1, 0, 1, 1, 0, 1, 0, 0], 4], 1: [[1, 0, 1, 1, 1, 1, 1, 1], 7], 2: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 3: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 4: [[1, 0, 1, 1, 1, 1, 1, 1], 7], 5: [[0, 1, 0, 1, 1, 1, 1, 1], 6], 6: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 7: [[0, 1, 0, 1, 1, 1, 1, 1], 6], 8: [[0, 0, 0, 1, 1, 1, 1, 1], 5], 9

Individual Functions

Whenever we update current population, we also update the heuristic value of each individual/state/chromosome. This is done using method updatePopulationFitness(). The individual fitness/heuristic value is calculated as simply the number of ones in an array of an individual. The goal is to maximize this value generation after generation.

In [ ]:
def updatePopulationFitness(self):
        self.totalFitness = 0
        for individual in self.population:
            individualFitness = sum(self.population[individual][0])
            self.population[individual][1] = individualFitness
            self.totalFitness = self.totalFitness + individualFitness

We now focus on how to select parents for reproduction. This is done using roulette wheel implementation. We first determine the size of the roulette wheel i.e., how many values will it store. This is simply set as 5 times the size of population, e.g., if we consider that we will retain 10 survivors at each generation then the size of the wheel is 50. We now have to fill these 50 values based upon the fitness value of each of those 10 individuals. This is done by calculating the probability of occurrence (h/sum(h)) of each individual. The variable individualLength determines how many values out of those 50 belong to that specific individual. Finally we generate random locations from roulette wheel populationSize times and select individuals based upon that. Those individuals now replace the original population.

In [ ]:
def selectParents(self):
    rouletteWheel = []
    wheelSize = self.populationSize * 5
    h_n = []
    for individual in self.population:
        h_n.append(self.population[individual][1])
    j = 0
    for individual in self.population:
        individualLength = round(wheelSize*(h_n[j]/sum(h_n)))
        j=j+1
        if individualLength > 0:
            i=0
            while i<individualLength:
                rouletteWheel.append(individual)
                i=i+1
    
    random.shuffle(rouletteWheel)
    parentIndices = []
    i=0
    while i<self.populationSize:
        parentIndices.append(rouletteWheel[random.randint(0, len(rouletteWheel)-1)])
        i=i+1
    newGeneration = dict()
    i=0
    while i<self.populationSize:
        newGeneration[i] = self.population[parentIndices[i]].copy()
        i=i+1
    del self.population
    self.population = newGeneration.copy()
    self.updatePopulationFitness()

We now come to the method generateChildren() which gerates children based upon crossover. A crossover probability is provided as input. For population size of 8 and crossover probability of 0.8 for instance, we need to do the crossover only between 80% of the 4 pairs i.e., we will do crossover only with 3 pairs and will take a pair as it is to next step (which is mutation). After doing crossover, we also update the fitness value of that child (although this shouldn’t be required since we are already calling updatePopulationFitness method at the end).

In [ ]:
def generateChildren(self, crossoverprobability):
    numberofPairs = round(crossoverprobability * self.populationSize / 2)
    individualIndices = list(range(0, self.populationSize))
    random.shuffle(individualIndices)
    i=0
    j=0
    while i<numberofPairs:
        crossoverPoint = random.randint(0, self.individualSize-1)
        child1 = self.population[j][0][0:crossoverPoint] + self.population[j+1][0][crossoverPoint:]
        child2 = self.population[j+1][0][0:crossoverPoint] + self.population[j][0][crossoverPoint:]
        self.population[j] = [child1, sum(child1)]
        self.population[j+1] = [child2, sum(child2)]
        i=i+2
        j=j+2
    self.updatePopulationFitness()

The next step is to mutate the population (not individual child) based upon a certain mutation probability provided as an input. For example, for individualSize of 5 bits and populationSize of 8 and a probability of 0.05, we need to swap 5% of those 8\*5=40 bits, i.e., round(0.05\*40) bits will be swapped.

In [ ]:
def mutateChildren(self, mutationProbability):
    numberOfBits = round(mutationProbability * self.populationSize *self.individualSize)
    totalIndices = list(range(0, self.populationSize*self.individualSize))
    random.shuffle(totalIndices)
    swapLocations = random.sample(totalIndices, numberOfBits)

    for loc in swapLocations:
        individualIndex = math.floor(loc/self.individualSize)
        bitIndex = math.floor(loc % self.individualSize)

        if self.population[individualIndex][0][bitIndex] == 0:
            self.population[individualIndex][0][bitIndex] = 1
        else:
            self.population[individualIndex][0][bitIndex] = 0
    self.updatePopulationFitness()

We now focus on our main function that will initialize the class. Whenever we find an individual in a generation which has ideal heuristic value, we terminate the algorithm.